# Stock Price Prediction - Petrobras S.A.

The aim of this project is to build machine learning models in order to predict future stock prices for the brazilian company Petrobras S.A. For this, we use three tree-based machine learning models: regression trees, random forest and XGBoost.

Authors: 

*Caio Lopes de Souza* 

*Gabriel Rodrigues Palma*

*Silvio Sandoval Zocchi*

## Packages used in this project

In [65]:
# data manipulation 
import numpy as np
import pandas as pd

# data extraction 
from yahooquery import Ticker 

# data visualization
import matplotlib.pyplot as plt

# machine learning models
from sklearn.ensemble import RandomForestRegressor 
from xgboost import XGBRegressor 

# models performance metrics
from sklearn.metrics import mean_squared_error

# preprocessing
from sklearn.preprocessing import StandardScaler

## Functions used in this project

In [66]:
def open_data(path):
    '''read a csv file and returns a dataframe'''
    
    dataframe = pd.read_csv(path, encoding='latin-1')
    
    return dataframe

In [67]:
def correct_data(dataframe, old_column, new_columns, split_point):
    '''correct data formatting'''
    
    dataframe[new_columns] = dataframe[old_column].str.split(split_point, expand=True)
    dataframe = dataframe.drop(old_column, axis=1)
    
    return dataframe

In [68]:
def correct_multindex(data, level, column_exclusion): 
    '''transform a multindex dataframe into a dataframe with only one index'''
    
    data = data.reset_index(level=level)
    data = data.drop(column_exclusion, axis=1)
    
    return data

In [69]:
def convert_datatypes(dataframe, column_to_numeric, column_to_date):
    '''change the datatype of two columns from object to numeric and date'''

    dataframe[column_to_numeric] = pd.to_numeric(dataframe[column_to_numeric])
    dataframe[column_to_date] = pd.to_datetime(dataframe[column_to_date], format='%d/%m/%Y')
    
    return dataframe

In [70]:
def line_plot(dataset, column_list, x_label, y_label, title):
    '''plot a standardized line plot for a chosen variable from a dataset'''
    color_list = ['blue', 'red', 'green']
    
    fig, ax = plt.subplots(figsize=(8, 3))
    for column in column_list: 
        dataset[column].plot(color=color_list[column_list.index(column)], ax=ax)
            
        ax.set_label(x_label)
        ax.set_ylabel(y_label)
        ax.set_title(title)
            
        plt.tight_layout()

In [71]:
def plot_test_error(y, predictions, model):
    '''plot the the real value of the response variable and its predictions by the model'''
    predictions = pd.Series(predictions)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    predictions.plot(ax=ax)
    
    y.plot(ax=ax)
    
    ax.legend(['Predictions', 'Actual Value'])
    
    ax.set_title(f"Comparison between the actual value of the response variable and its correspondent prediction for the {model}",)
    
    ax.set_xlabel('Date')
    ax.set_ylabel('Value/Prediction')

    plt.tight_layout()

In [72]:
def training_test_split(X, y, starting_point = 0.8):
    '''split a dataset into a training and test set for a chosen ratio'''
    X_train = X.iloc[:round(starting_point * len(X))]
    X_test = X.iloc[round(starting_point * len(X)):]
    y_train = y.iloc[:round(starting_point * len(y))]
    y_test = y.iloc[round(starting_point * len(y)):]
    
    return X_train, X_test, y_train, y_test

In [73]:
def one_step_ahead_forecasting(X, y, model, starting_point):
    '''predict the next observation of a time series using as training set all the past observations'''
    
    predictions = np.array([])
    
    X_train, X_test, y_train, y_test = training_test_split(X, y, starting_point = starting_point)
    model.fit(X_train.values, y_train.values)
    
    for i in range(len(X_test)):
        predictions = np.append(predictions, (model.predict(X_test.iloc[i].values.reshape(1, -1))))
        
        X_train = pd.concat([X_train, X_test.iloc[i].to_frame().T])
        y_train = np.append(y_train, y_test.iloc[i].reshape(1, -1))
        
        y_train = pd.Series(y_train)
        
        model.fit(X_train.values, y_train.values)
        
    return predictions

In [74]:
def one_step_ahead_forecasting_standard(X, y, model, starting_point):
    '''predict the next observation of a time series using as training set all the past observations'''
    
    predictions = np.array([])
    
    X_train, X_test, y_train, y_test = training_test_split(X, y, starting_point = starting_point)
    
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    
    X_test = scaler.transform(X_test)
 
    model.fit(X_train, y_train.values)
    
    for i in range(len(X_test)):
        X_test.iloc[i] = scaler.transform(X_test.iloc[i])
        
        predictions = np.append(predictions, (model.predict(X_test.iloc[i].values.reshape(1, -1))))
        
        X_train = pd.concat([X_train, X_test.iloc[i].to_frame().T])
        y_train = np.append(y_train, y_test.iloc[i].reshape(1, -1))
        
        y_train = pd.Series(y_train)
        
        model.fit(X_train, y_train.values)
        
    return predictions

## Data gathering and manipulation

###  Stock Prices

#### Data Gathering

In [75]:
stock = Ticker("PETR4.SA")
stock = stock.history(period="max")

In [76]:
stock_price = stock['close']
stock_price = pd.DataFrame(stock_price, columns =['close'])

#### Data manipulation

In [77]:
stock_price = correct_multindex(stock_price, level=[0, 1], column_exclusion = 'symbol')

In [78]:
stock_price = stock_price.drop(stock_price.tail(1).index)

In [79]:
stock_price = convert_datatypes(stock_price, column_to_numeric = 'close', column_to_date='date')

In [80]:
stock_price = stock_price.rename(columns={"date": "Date", "close": "Close_price"})

In [81]:
stock_price

,Date,Close_price
0,2000-01-03,5.875000
1,2000-01-04,5.550000
2,2000-01-05,5.494000
3,2000-01-06,5.475000
4,2000-01-07,5.500000
...,...,...
5948,2023-09-05,33.369999
5949,2023-09-06,33.520000
5950,2023-09-08,33.400002
5951,2023-09-11,33.369999


### Exchange rate 

Here, we use the commercial free float exchange rates between the brazilian Real and the american Dollar. It shows the a daly data between 11/28/1984 and 06/30/2023. The brazilian Central Bank computes this rate through the refference rate "PTAX".

* PTAX Methodology: http://www.ipeadata.gov.br/doc/EE042_A_taxa_de_cambio_de_referencia_Ptax.pdf

* Link for the data source: https://www3.bcb.gov.br/sgspub/consultarvalores/consultarValoresSeries.do?method=getPagina 

#### Data gathering

In [82]:
exchange_rate = open_data(r"C:\Users\caiol\Petrobras S.A\Dados_input\Câmbio_diário(1984-2023)_v2.csv")

#### Data manipulation 

In [83]:
exchange_rate = correct_data(exchange_rate, 'Data;Cambio', ['Date', 'Exchange_rate'], ';')

In [ ]:
exchange_rate

In [ ]:
exchange_rate = convert_datatypes(exchange_rate, column_to_numeric = 'Exchange_rate', column_to_date = 'Date')

In [ ]:
exchange_rate.head()

### Interest Rates (SELIC)

The most basic and important interest rate in Brazil is the SELIC rate, which is the rediscount rate for the brazilian economy and has an influence over all other interest rates in the country.

#### Data gathering

In [ ]:
selic = open_data(r"C:\Users\caiol\Petrobras S.A\Dados_input\SELIC(2000-2023).csv")

#### Data manipulation

In [ ]:
selic = correct_data(selic, 'Data;Selic', ['Date', 'Selic'], ';')

In [ ]:
selic = convert_datatypes(selic, column_to_numeric = 'Selic', column_to_date = 'Date')

In [ ]:
selic.head()

### Oil Price

#### Data gathering

In [ ]:
oil = open_data(r"C:\Users\caiol\Petrobras S.A\Dados_input\petróleo_preço(1987-2023).csv")

#### Data manipulation

In [ ]:
oil = correct_data(oil, 'Data;Preço do petróleo brent(dólares/barril)', ['Date', 'Oil_price'], ';')

In [ ]:
oil = convert_datatypes(oil, column_to_numeric = 'Oil_price', column_to_date = 'Date')

In [ ]:
oil.head()

### Merging all together

In [ ]:
data_list = [stock_price, exchange_rate, selic, oil]

In [ ]:
dataset = stock_price.merge(exchange_rate, on='Date')
dataset = dataset.merge(selic, on='Date')
dataset = dataset.merge(oil, on='Date')

In [ ]:
dataset.index = dataset['Date']

dataset = dataset.drop('Date', axis=1)

In [ ]:
dataset.head()

In [ ]:
dataset.to_csv('petrobras_data.csv')

### Exploratory Data Analysis

In [ ]:
font = {
    'family': 'serif',
    'color': 'black',
    'weight': 'normal',
    'size': 10
}

In [ ]:
def line_plot(dataset, column_list, x_label, y_label, title):
    color_list = ['blue', 'red', 'green']
    
    fig, ax = plt.subplots(figsize=(8, 3))
    for column in column_list: 
        dataset[column].plot(color=color_list[column_list.index(column)], ax=ax)
            
        ax.set_label(x_label)
        ax.set_ylabel(y_label)
        ax.set_title(title)
            
        plt.tight_layout()

#### Data Visualization

In [ ]:
line_plot(dataset, column_list = ['Close_price'], x_label='Time (days)', y_label='Stock Price',
          title = 'Stock Price of Petrobras S.A. (2000-2023)')

In [ ]:
plt.figure(figsize=(15, 10))

for i, data in enumerate(dataset.columns, 1):
    plt.subplot(2, 2, i)
    dataset[data].plot()
    plt.xlabel("Date")
    plt.ylabel("Value")
    plt.title(f"{data} time series")
plt.tight_layout()

### Training-test Split

In [ ]:
X = dataset.drop('Close_price', axis=1)
y = dataset['Close_price']

In [ ]:
X_train, X_test, y_train, y_test = training_test_split(X, y)

### A naive model 

For a naive model, which will be used to compare the performance of the other models, we'll consider the value of the estimate for the response variable as being equal to the value of the response variable in the past day. In other words,  $\hat{y}_{t+1} = y_{t}$

In [ ]:
dataset['price_lag_1'] = dataset['Close_price'].shift(1)
dataset = dataset.drop(dataset.head(1).index, axis=0)

In [ ]:
X_lag = dataset['price_lag_1']
y_lag = dataset['Close_price']

In [ ]:
X_lag_2 = pd.DataFrame(X_lag)
y_lag_2 = pd.DataFrame(y_lag) 

In [ ]:
lag = pd.concat([X_lag_2, y_lag_2])
lag

In [ ]:
X_train_lag, X_test_lag, y_train_lag, y_test_lag = training_test_split(X_lag, y_lag)

In [ ]:
mean_squared_error(X_test_lag, y_test_lag)

In [ ]:
plot_test_error(y = y_test, predictions = y_test_lag, model = "Regression Tree")

In [ ]:
dataset = dataset.drop('price_lag_1', axis=1)

#### Fixing the dataset

In [ ]:
dataset['Close_price'] = dataset['Close_price'].shift(1)
dataset = dataset.dropna()

In [ ]:
y = dataset['Close_price']
X = dataset.drop('Close_price', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = training_test_split(X, y)

#### Regression Tree (multiple starting points)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
reg_tree = DecisionTreeRegressor(random_state=42)

In [ ]:
tree_results = {}

for starting_point in range(20, 100, 20):
    X_train, X_test, y_train, y_test = training_test_split(X, y, starting_point = (starting_point/100))
    tree_prediction = one_step_ahead_forecasting(X = X, y = y, model = reg_tree,
                                                       starting_point = (starting_point/100))
    
    
    tree_results[starting_point] = mean_squared_error(y_test, tree_prediction)

In [ ]:
tree_results

### Regression Tree

In [ ]:
X_train, X_test, y_train, y_test = training_test_split(X, y, starting_point = 0.6)

In [ ]:
tree_predictions = one_step_ahead_forecasting(X = X, y = y, model = reg_tree ,starting_point=0.6)

In [ ]:
tree_predictions = pd.Series(tree_predictions, index = y_test.index)

In [ ]:
print(f"The MSE for the model is {mean_squared_error(y_test, tree_predictions)}")

In [ ]:
plot_test_error(y = y_test, predictions = tree_predictions, model = "Regression Tree")

In [ ]:
def residual_plot(y_test, predictions):
    residual = predictions - y_test

    fig, ax = plt.subplots(figsize=(10,6))
    
    residual.plot(ax=ax)
    
    

In [ ]:
residual_plot(y_test, tree_predictions)

### Random Forest

In [ ]:
rf_reg = RandomForestRegressor(max_depth = 10, random_state=42)

In [ ]:
rf_predictions = one_step_ahead_forecasting(X = X, y = y, model = rf_reg, starting_point = 0.6)

In [ ]:
mean_squared_error(y_test, rf_predictions)

In [ ]:
rf_predictions = pd.Series(rf_predictions, index=y_test.index)

### Grid Search for Random Forest

In [ ]:
n_estimators = [i for i in range(100, 1000, 100)]
max_depth = [i for i in range(10, 100, 10)]
min_samples_split = [i for i in range(2, 12, 2)]

rf_parameters = {}

#### MSE

In [ ]:
print(f"The MSE for the model is {mean_squared_error(y_test, rf_predictions)}")

#### Performance visualization

In [ ]:
plot_test_error(y = y_test, predictions = rf_predictions, model = "Random Forest")

#### Random Forest (multiple starting points)

In [ ]:
rf_results = {}

for starting_point in range(20, 100, 20):
    X_train, y_train, X_test, y_test = training_test_split(X, y, starting_point = (starting_point/100))
    rf_prediction = one_step_ahead_forecasting(X = X, y = y, model = rf_reg,
                                                       starting_point = (starting_point/100))
    
    
    rf_results[starting_point] = mean_squared_error(y_test, rf_prediction)

In [ ]:
rf_results

### XGBoost

In [ ]:
xgb = XGBRegressor()

#### Fit

In [ ]:
xgb_predictions = one_step_ahead_forecasting(X = X, y = y, model = xgb, starting_point = 0.6)

In [ ]:
xgb_predictions = pd.Series(xgb_predictions, index=X_test.index)

In [ ]:
mean_squared_error(y_test, xgb_predictions)

#### Performance visualization

In [ ]:
plot_test_error(y = y_test, predictions = xgb_predictions, model = "XGBoost")

#### XGBoost (multiple starting points)

In [ ]:
xgb_results = {}

for starting_point in range(20, 100, 20):
    X_train, y_train, X_test, y_test = training_test_split(X, y, starting_point = (starting_point/100))
    xgb_prediction = one_step_ahead_forecasting(X = X, y = y, model = xgb,
                                                       starting_point = (starting_point/100))
    
    
    xgb_results[starting_point] = mean_squared_error(y_test, xgb_prediction)

### Adding lags

In [ ]:
for i in range(1, 9, 2):
    dataset[f"price_lag_{i}"] = dataset['Close_price'].shift(i)
dataset = dataset.dropna(axis=0)

In [ ]:
dataset.head()

In [ ]:
y = dataset['Close_price']
X = dataset.drop('Close_price', axis=1)

X_only_lag = X.drop(['Exchange_rate', 'Selic', 'Oil_price'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = training_test_split(X, y, starting_point = 0.6)

### Regression Tree with Lag

In [ ]:
tree_lag_predictions = one_step_ahead_forecasting(X = X, y = y, model = reg_tree ,starting_point=0.6)

In [ ]:
tree_lag_predictions = pd.Series(tree_lag_predictions, index = y_test.index)

In [ ]:
mean_squared_error(y_test, tree_lag_predictions)

### Regression tree only lags

In [ ]:
X = X.drop(['Exchange_rate', 'Selic', 'Oil_price'], axis=1)

In [ ]:
tree_only_lag_predictions = one_step_ahead_forecasting(X = X_only_lag, y = y, model = reg_tree, starting_point = 0.6)

In [ ]:
tree_only_lag_predictions = pd.Series(tree_only_lag_predictions, index = y_test.index)

In [ ]:
mean_squared_error(y_test, tree_only_lag_predictions)

### Random Forest with Lags

In [ ]:
rf_lag_predictions = one_step_ahead_forecasting(X = X, y = y, model = rf_reg, starting_point = 0.6)

In [ ]:
rf_lag_predictions = pd.Series(rf_lag_predictions, index = y_test.index)
mean_squared_error(y_test, rf_lag_predictions)

### Random Forest only with Lags

In [ ]:
rf_only_lag_predictions = one_step_ahead_forecasting(X = X_only_lag, y = y,
                                                     model = rf_reg, starting_point = 0.6)

In [ ]:
rf_only_lag_predictions = pd.Series(rf_only_lag_predictions, index = y_test.index)
mean_squared_error(y_test,)

### Xgboost with lag

In [ ]:
xgb_lag_predictions = one_step_ahead_forecasting(X = X, y = y, model = xgb, starting_point = 0.6)

In [ ]:
xgb_lag_predictions = pd.Series(xgb_lag_predictions, index = y_test.index)
mean_squared_error(y_test, xgb_lag_predictions)

### XGBoost only with lags

In [ ]:
xgb_only_lag_predictions = one_step_ahead_forecasting(X = X_only_lag, y = y_only_lag, 
                                                      model = xgb, starting_point = 0.6)

In [ ]:
xgb_only_lag_predictions = pd.Series(xgb_only_lag_predictions, index = y_test.index)
mean_squared_error(y_test, xgb_only_lag_predictions)